https://api.nasa.gov/

In [ ]:
import requests
from datetime import datetime, timedelta
import time
import os
from dotenv import load_dotenv

load_dotenv()

# Configuration Section
class Config:
    # Base URL for the NASA Mars Photos API
    NASA_MARS_PHOTOS_BASE_URL = "https://api.nasa.gov/mars-photos/api/v1/rovers"
    
    # Make sure you include a <.env> file in the same dir as this script. That file should contain: NASA_API_KEY=0d8784f64eb949398c7b427a5d30a0da
    API_KEY = os.getenv("NASA_API_KEY")

    DEFAULT_RATE_LIMIT = 1.0
    
    CAMERAS = [
        "MCZ_RIGHT", "MCZ_LEFT", "NAVCAM_LEFT", "NAVCAM_RIGHT"
    ]

    # Mapping of cameras to the rovers that support them:
    CAMERA_ROVER_MAPPING = { cam.lower(): ["perseverance"] for cam in CAMERAS }
    
    # Rover landing dates (used to calculate the Martian sol from an Earth date)
    ROVER_LANDING_DATES = {
        "curiosity": "2012-08-06",
        "opportunity": "2004-01-25",
        "spirit": "2004-01-04",
        "perseverance": "2021-02-18"
    }
    
    # Default Earth date range (in YYYY-MM-DD format) for queries
    DEFAULT_START_DATE = "2022-02-20"
    DEFAULT_END_DATE = "2025-02-21"
    
    # Local directory where images will be downloaded
    DOWNLOAD_DIR = "mars_images"
    
    # Set to True to download images, or False to just print URL and API response info
    DOWNLOAD_IMAGES = True

# Helper Functions
def daterange(start_date, end_date):
    """Generate dates from start_date to end_date (inclusive)."""
    for n in range((end_date - start_date).days + 1):
        yield start_date + timedelta(n)

def download_image(url, directory):
    """Download an image from the given URL to the specified directory.
       Returns the local file path if successful, or None if not."""
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = os.path.basename(url)
    filepath = os.path.join(directory, filename)
    print(f"Downloading image from: {url}")
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Downloaded image to: {filepath}")
            return filepath
        else:
            print(f"Failed to download image. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None

def send_requests():
    # Use configuration values
    start_date_str = Config.DEFAULT_START_DATE
    end_date_str = Config.DEFAULT_END_DATE
    api_key = Config.API_KEY
    rate_limit = Config.DEFAULT_RATE_LIMIT
    download_flag = Config.DOWNLOAD_IMAGES

    # Parse dates
    try:
        start_date = datetime.strptime(start_date_str, "%Y-%m-%d").date()
        end_date = datetime.strptime(end_date_str, "%Y-%m-%d").date()
    except ValueError:
        print("Invalid date format. Please use YYYY-MM-DD.")
        return

    if start_date > end_date:
        print("'start_date' cannot be later than 'end_date'.")
        return

    results = []   # To aggregate photo data
    rate_info = [] # To store rate-limit headers from each NASA API response

    # Calculate delay between requests
    delay = 1.0 / rate_limit if rate_limit > 0 else 0

    # Iterate through each Earth date in the range
    for current_date in daterange(start_date, end_date):
        earth_date_str = current_date.strftime("%Y-%m-%d")
        for camera in Config.CAMERAS:
            for rover in Config.CAMERA_ROVER_MAPPING.get(camera.lower(), []):
                # Calculate the Martian sol: days since the rover landed
                landing_date = datetime.strptime(Config.ROVER_LANDING_DATES[rover], "%Y-%m-%d").date()
                sol = (current_date - landing_date).days
                if sol < 0:
                    # Skip Earth dates before the rover landed
                    continue

                # Build the URL and query parameters
                url = f"{Config.NASA_MARS_PHOTOS_BASE_URL}/{rover}/photos"
                params = {
                    "sol": sol,
                    "camera": camera,
                    "api_key": api_key
                }
                # Construct the full URL with parameters and print it
                full_url = requests.Request('GET', url, params=params).prepare().url
                print(f"Constructed URL: {full_url}")
                print(f"Sending request to: {url} with parameters: {params}")
                
                response = requests.get(url, params=params)
                print(f"Received response from: {url} with status code: {response.status_code}")
                
                # Record rate-limit headers from this request
                rate_info.append({
                    "earth_date": earth_date_str,
                    "rover": rover,
                    "camera": camera,
                    "sol": sol,
                    "X-RateLimit-Limit": response.headers.get("X-RateLimit-Limit"),
                    "X-RateLimit-Remaining": response.headers.get("X-RateLimit-Remaining")
                })
                
                if response.status_code == 200:
                    data = response.json()
                    if "photos" in data:
                        for photo in data["photos"]:
                            # Tag photo with source details
                            photo["queried_rover"] = rover
                            photo["queried_camera"] = camera
                            photo["queried_sol"] = sol
                            photo["queried_earth_date"] = earth_date_str
                            
                            # If download flag is set, download the image and add its local path
                            if download_flag:
                                img_url = photo.get("img_src")
                                if img_url:
                                    local_path = download_image(img_url, Config.DOWNLOAD_DIR)
                                    photo["local_path"] = local_path
                        results.extend(data["photos"])
                else:
                    results.append({
                        "earth_date": earth_date_str,
                        "rover": rover,
                        "camera": camera,
                        "sol": sol,
                        "error": response.json()
                    })

                # Pause between requests
                time.sleep(delay)
    
    # Print out the aggregated results and rate limit info
    print("\n=== Aggregated Results ===")
    print(results)
    print("\n=== Rate Limit Information ===")
    print(rate_info)

send_requests()
